# SoftMax

In [36]:
context= [100,20,30]
temperature=1
n_arms=15

In [37]:
weights=[[0, 0.7, 0.69],
 [0.72, 0.72, 0.72],
 [0.72, 0.72, 0.72],
 [0.72, 0.72, 10],
 [0.72, 0.72, 0.72],
 [0.72, 0.72, 0.72],
 [0.72, 0.72, 0.72],
 [0.72, 0.72, 0.72],
 [0.72, 0.72, 0.72],
 [0.72, 0.72, 0.72],
 [0.72, 0.72, 0.72],
 [0.72, 0.72, 0.72],
 [0.72, 0.72, 0.72],
 [0.72, 0.72, 0.72],
 [1, 0.87, 1]]

In [44]:
def soft_max(context, temperature, n_arms, weights):
    context_np = np.array(context).reshape(1, -1)  # Convertir a fila vector

    context_np = context_np / np.sum(context_np)  # Normalización del contexto

    expected_rewards = np.dot(weights, context_np.T).flatten()

    # Aplicar softmax
    exp_rewards = np.exp(
        (expected_rewards - np.max(expected_rewards)) / temperature
    ) 

    sum_exp_rewards = np.sum(exp_rewards)
    if sum_exp_rewards == 0:
        choice = np.random.choice(n_arms)
    else:
        try:
            probabilities = exp_rewards / np.sum(exp_rewards)
            choice = np.random.choice(n_arms, p=probabilities)
        except Exception as e:
            print(e)
            print(weights)
            print(probabilities)
            choice = np.random.choice(n_arms)
    print(exp_rewards)
    return choice
    

In [46]:
soft_max(context, temperature, n_arms, weights)

[0.09587916 0.15629657 0.15629657 1.         0.15629657 0.15629657
 0.15629657 0.15629657 0.15629657 0.15629657 0.15629657 0.15629657
 0.15629657 0.15629657 0.20324699]


7

In [47]:
softmax_choice(context, temperature, n_arms, weights)

[0.09587916 0.15629657 0.15629657 1.         0.15629657 0.15629657
 0.15629657 0.15629657 0.15629657 0.15629657 0.15629657 0.15629657
 0.15629657 0.15629657 0.20324699]


3

In [43]:
exp_rewards

array([0.09587916, 0.15629657, 0.15629657, 1.        , 0.15629657,
       0.15629657, 0.15629657, 0.15629657, 0.15629657, 0.15629657,
       0.15629657, 0.15629657, 0.15629657, 0.15629657, 0.20324699])

In [8]:
choice

7

# Velocidad trozos algoritmo

In [3]:
import pandas as pd
import numpy as np

In [4]:
def add_solutions(solution, f1, f2, f3, route_solutions, df_solutions):
    solution = str(sorted(solution))
    solucion_encontrada = False

    if not df_solutions.empty:
        if solution in df_solutions["solution"].values:
            return  # La solución ya existe, no hacer nada

        df_dominado = df_solutions.copy()
        df_dominado = df_dominado[df_dominado["f1"] <= f1]
        df_dominado = df_dominado[df_dominado["f2"] <= f2]
        df_dominado = df_dominado[df_dominado["f3"] <= f3]
        if df_dominado.empty:
            # Eliminar soluciones que sean dominadas por la nueva
            df_solutions = df_solutions[
                ~(
                    (df_solutions["f1"] >= f1)
                    & (df_solutions["f2"] >= f2)
                    & (df_solutions["f3"] >= f3)
                    & (
                        (df_solutions["f1"] > f1)
                        | (df_solutions["f2"] > f2)
                        | (df_solutions["f3"] > f3)
                    )
                )
            ]
            # Agregar la nueva solución
            new_solution = pd.DataFrame(
                [{"solution": solution, "f1": f1, "f2": f2, "f3": f3}]
            )
            df_solutions = pd.concat([df_solutions, new_solution], ignore_index=True)
            df_solutions.to_csv(route_solutions, index=False)
            solucion_encontrada = True
    else:
        df_solutions = pd.DataFrame(
            [{"solution": solution, "f1": f1, "f2": f2, "f3": f3}]
        )
        df_solutions.to_csv(route_solutions, index=False)
    return solucion_encontrada

In [2]:
def add_solutions_optimized(solution, f1, f2, f3, route_solutions, df_solutions):
    """
    Versión optimizada que utiliza NumPy para comprobaciones de dominancia
    y un set para verificar duplicados rápidamente.
    """
    solution = str(sorted(solution))

    # --- CASO BASE: Si el DataFrame está vacío, añadir y salir ---
    if df_solutions.empty:
        df_solutions = pd.DataFrame(
            [{"solution": solution, "f1": f1, "f2": f2, "f3": f3}]
        )
        df_solutions.to_csv(route_solutions, index=False)
        return True

    # Usar un set para comprobar existencia (O(1) en promedio) ---
    existing_solutions = set(df_solutions["solution"])
    if solution in existing_solutions:
        return False # La solución ya existe


    # Extraer los valores numéricos a un array de NumPy para cálculos rápidos
    vals = df_solutions[["f1", "f2", "f3"]].values
    new_vals = np.array([f1, f2, f3])


    if np.any(np.all(vals <= new_vals, axis=1)):
        return False # La nueva solución es dominada, no hacer nada

    # 2. Si no es dominada, eliminar las soluciones que SÍ son dominadas por la nueva.
    #    Una solución 'new' domina a una existente 's' si:
    #    (new_f1 <= s_f1, new_f2 <= s_f2, new_f3 <= s_f3) Y
    #    (new_f1 < s_f1  O  new_f2 < s_f2  O  new_f3 < s_f3)
    #    Lo calculamos con una máscara booleana.
    
    # Condición 1: Todos los valores de la nueva solución son menores o iguales
    # a los de las soluciones existentes.
    cond1 = np.all(new_vals <= vals, axis=1)
    # Condición 2: Al menos un valor de la nueva solución es estrictamente menor.
    cond2 = np.any(new_vals < vals, axis=1)
    
    # La máscara final identifica las filas a eliminar (las dominadas)
    dominated_mask = cond1 & cond2
    
    # Mantener solo las filas que NO están dominadas
    df_solutions = df_solutions[~dominated_mask]

    # 3. Añadir la nueva solución no dominada
    new_solution_df = pd.DataFrame(
        [{"solution": solution, "f1": f1, "f2": f2, "f3": f3}]
    )
    df_solutions = pd.concat([df_solutions, new_solution_df], ignore_index=True)
    
    # Guardar el resultado final en el CSV
    df_solutions.to_csv(route_solutions, index=False)
    
    return True

In [1]:
import os

In [15]:
archive = "WorkSpace 1000_50_5"
solution=[5, 8, 11, 27, 40]
f1_value=200
f2_value=300
f3_value=100

In [6]:
folder_solutions = f"Solutions/Multiprocessing/{archive}/"

route_solutions = folder_solutions + archive + ".csv"

if os.path.exists(route_solutions):
    df_solutions = pd.read_csv(route_solutions)

In [16]:

add_solutions(solution, f1_value, f2_value, f3_value, route_solutions, df_solutions)

True

In [17]:

add_solutions_optimized(solution, f1_value, f2_value, f3_value, route_solutions, df_solutions)

True